In [1]:
import re
import string

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import gensim.downloader as api

import session_info

In [2]:
# Посмотрим версии используемых модулей.
session_info.show()

In [2]:
# Загружаем из nltk стоп слова для русского и английского языка
stop1 = list(stopwords.words('english'))
stop2 = list(stopwords.words('russian'))
stop = stop1 + stop2

lemmatizer = WordNetLemmatizer()

# Функция для предобработки. Удаление знаков пуктуации, приведение к нижнему регистру, лемматизация.
def preprocessing(line):
    line = line.lower()
    line = re.sub(r'[.,"\'-?:!;]', "", line)
    line = nltk.word_tokenize(line)
    line = ' '.join([lemmatizer.lemmatize(w) for w in line])
    return line

In [3]:
# Читаем наш датасет.
df = pd.read_csv('train.csv')

In [4]:
# Посмотрим на данные.
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497819 entries, 0 to 497818
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   pair_id       497819 non-null  int64 
 1   name_1        497819 non-null  object
 2   name_2        497819 non-null  object
 3   is_duplicate  497819 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 15.2+ MB


In [6]:
# Посмотрим сколько одинаковых пар, а сколько разных.
df.is_duplicate.value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

In [7]:
# Разбиваем датафрейм на на части с копиями и без них.
df_0 = df[df['is_duplicate'] == 0]
df_1 = df[df['is_duplicate'] == 1]

name_1_df_0 = list(df_0['name_1'])
name_2_df_0 = list(df_0['name_2'])

name_1_df_1 = list(df_1['name_1'])
name_2_df_1 = list(df_1['name_2'])

In [8]:
# Объединим два столбца, посмотрим сколько всего названий.
text = list(df['name_1']) + list(list(df['name_2']))

print(len(text))

995638


In [9]:
# Оставим только уникальные названия.
text_unic = list(set(text))

print("Количество уникальных названий  " , len(text_unic))

# Разобъем все названия на отдельные слова
words = []
for string in text_unic:
    subwords = preprocessing(string).split()
    for word in subwords:
        words.append(word)

print("Количество слов" , len(words))
words_unic = list(set(words))
print("Количество уникальных слов ", len(words_unic))

Количество уникальных названий   18022
Количество слов 73720
Количество уникальных слов  16046


In [10]:
print(preprocessing(text[12]))

bestocean worldwide logistics inc


In [11]:
# Оставим только слова длиннее 1 символа.
filtered_words = [w for w in words_unic if len(w) > 1]

# Загрузим предобученную модель word2vec на полтора гигабайта
word2vec = api.load("word2vec-google-news-300")

In [12]:
# Функция делает эмбеддинг из строки, как сумму эмбеддингов входящих в строку слов
def sentence_embedding(string):
    sentence_list = [word2vec.get_vector(w) if w in word2vec else np.zeros(300) for w in string.split()  ]
    result = np.sum(np.array(sentence_list), axis=0)
    return result

In [13]:
# Пример текста. Предобработаем его, вычислим эмбеддинги, а потом посмотрим на косинусное сходство.
text1 = preprocessing('Carlisle Coatings & Waterproofing, Inc.' )
text2 = preprocessing('Carlisle Coatings & Wtrprfng')

print(text1)
print(text2)

text1_vec = sentence_embedding(text1)
text2_vec = sentence_embedding(text2)

carlisle coating & waterproofing inc
carlisle coating & wtrprfng


In [14]:
# Посмотрим на косинусное сходство двух текстов.
print(cosine_similarity(text1_vec.reshape(1, -1), text2_vec.reshape(1, -1)))
print(cosine_similarity(text1_vec.reshape(1, -1), text2_vec.reshape(1, -1))[0][0])

[[0.84854585]]
0.8485458487282259


In [15]:
# Вычислим косинусное сходство для всех пар из нашего датасета.
pair_0 = [cosine_similarity(sentence_embedding(item[0]).reshape(1, -1), sentence_embedding(item[1]).reshape(1, -1)) for item in zip(name_1_df_0 ,name_2_df_0 )]
pair_1 = [cosine_similarity(sentence_embedding(item[0]).reshape(1, -1), sentence_embedding(item[1]).reshape(1, -1)) for item in zip(name_1_df_1 ,name_2_df_1 )]  

In [16]:
# Посмотрим на косинусное сходство первых 50 непохожиш пар.
print(pair_0[:50])

[array([[0.50184368]]), array([[0.98082832]]), array([[0.]]), array([[0.33745104]], dtype=float32), array([[0.12854507]]), array([[0.84596272]]), array([[0.33577943]], dtype=float32), array([[0.]]), array([[0.28325106]]), array([[0.34646974]]), array([[0.]]), array([[0.54023613]]), array([[0.69775787]]), array([[0.54692054]]), array([[0.12427095]], dtype=float32), array([[0.]]), array([[0.37939467]]), array([[0.57596859]]), array([[0.09194427]]), array([[0.]]), array([[0.08875784]]), array([[0.48656179]]), array([[0.56370787]]), array([[0.626007]]), array([[0.]]), array([[0.33055593]]), array([[0.29227863]]), array([[0.31663013]]), array([[0.61071167]]), array([[0.]]), array([[0.53513233]]), array([[0.817649]], dtype=float32), array([[0.6009203]], dtype=float32), array([[0.38137397]]), array([[0.36928403]]), array([[0.27060674]]), array([[0.45668453]], dtype=float32), array([[0.23539401]]), array([[0.30541746]]), array([[0.19433094]]), array([[0.27887128]]), array([[0.145431]]), array(

In [17]:
# Посмотрим на косинусное сходство первых 50 похожиш пар.
print(pair_1[:50])

[array([[0.902885]]), array([[0.74152608]]), array([[0.75264038]]), array([[0.27097568]]), array([[0.71816032]]), array([[0.45783442]]), array([[0.70380785]]), array([[0.93016223]]), array([[0.61920326]]), array([[0.4405077]]), array([[0.78023845]], dtype=float32), array([[0.6673595]], dtype=float32), array([[0.72691861]]), array([[0.84197215]]), array([[0.60947966]]), array([[0.94203689]]), array([[0.]]), array([[0.49931039]]), array([[0.38948551]]), array([[0.89311576]], dtype=float32), array([[0.36861226]], dtype=float32), array([[0.67940065]]), array([[0.81016271]]), array([[0.64658076]], dtype=float32), array([[0.43093696]]), array([[0.82132138]]), array([[1.]]), array([[0.81036796]]), array([[0.]]), array([[0.76877966]]), array([[0.70892515]]), array([[0.4290575]], dtype=float32), array([[0.78059737]]), array([[0.]]), array([[0.]]), array([[0.5575573]]), array([[0.58716923]], dtype=float32), array([[0.61940515]], dtype=float32), array([[0.27922883]]), array([[1.]]), array([[0.799

In [18]:
print(len(pair_0))
print(len(pair_1))

494161
3658


In [19]:
# Выведем долю пар схожих и отличных по косинусному сходству по порогу 0.3
pair_0_trashold_05 = [i for i in pair_0 if i < 0.3]
pair_1_trashold_05 = [i for i in pair_1 if i>= 0.3]

print(len(pair_0_trashold_05), "  ",len(pair_0_trashold_05)/len(pair_0)*100)

print(len(pair_1_trashold_05), "  ",len(pair_1_trashold_05)/len(pair_1)*100)

201255    40.72660529665433
2878    78.6768726079825
